In [2]:
import pandas as pd
fields = ['PPSORT', 'WEIGHT', 'ABOID', 'BFNMEMB', 'CFSTAT', 'Citizen', 'CMA',
       'DETH123', 'HHTYPE',
       'MarStH', 'PR', 'PRIHM', 'REGIND', 'Sex', 'SHELCO',
       'AGEGRP', 'ATTSCH', 'BedRm', 'CFInc', 'CFInc_AT', 'CfSize',
       'CIP2011', 'CIP2011_STEM_SUM', 'CONDO', 'DPGRSUM', 'DTYPE',
       'EFDecile', 'EfDIMBM', 'EFInc', 'EFInc_AT', 'EfSize', 'ETHDER',
       'GENSTAT', 'HCORENEED_IND', 'HDGREE', 'HHInc', 'HHInc_AT',
       'HHMRKINC', 'HHSIZE', 'HLANO', 'IMMCAT5', 'IMMSTAT',
       'LFACT', 'LICO', 'LICO_AT', 'LOC_ST_RES', 'LoLIMA', 'LoLIMB',
       'LoMBM', 'LSTWRK', 'MOB1', 'Mob5', 'MrkInc',
       'MTNNO', 'NOS', 'PKID0_1', 'PKID15_24', 'PKID2_5', 'PKID25',
       'PKID6_14', 'PKIDS', 'POB', 'POBF', 'POBM', 'PR1', 'PR5',
       'PresMortG', 'REPAIR', 'ROOMS', 'SSGRAD', 'Tenur', 'TotInc',
       'TotInc_AT', 'VALUE', 'VisMin', 'WRKACT']
dataset = pd.read_csv("pumf-2016-selected-features.csv", usecols=fields)
dataset.head()

,PPSORT,WEIGHT,ABOID,BFNMEMB,CFSTAT,Citizen,CMA,DETH123,HHTYPE,MarStH,...,PresMortG,REPAIR,ROOMS,SSGRAD,Tenur,TotInc,TotInc_AT,VALUE,VisMin,WRKACT
0,453141,37.037277,6,0,2,1,505,1,2,2,...,1,1,11,5,1,97000,73000,450000,13,11
1,923226,37.037277,6,0,4,1,505,2,2,1,...,1,1,11,4,1,49071,39759,440000,13,11
2,385097,37.037277,6,0,4,1,505,2,2,1,...,1,1,11,4,1,49071,39759,440000,13,11
3,732612,37.037277,6,0,2,1,999,2,2,3,...,1,2,8,8,1,46000,41000,839779,13,11
4,143665,37.120914,6,0,1,1,999,1,1,3,...,1,3,5,6,1,30000,26000,60000,13,9


In [3]:
motherTongueDict = {}
motherTongueDict['1'] = 'No non-official languages'
motherTongueDict['2'] = 'Aboriginal languages'
motherTongueDict['3'] = 'Arabic'
motherTongueDict['4'] = 'Mandarin'
motherTongueDict['5'] = 'Cantonese'
motherTongueDict['6'] = 'Chinese languages'
motherTongueDict['7'] = 'German'
motherTongueDict['8'] = 'All other languages' #'Other Germanic languages'
motherTongueDict['9'] = 'All other languages' #'Greek'
motherTongueDict['10'] = 'Urdu'
motherTongueDict['11'] = 'Persian (Farsi)'
motherTongueDict['12'] = 'Other Indo-Iranian languages'
motherTongueDict['13'] = 'Italian'
motherTongueDict['14'] = 'Polish'
motherTongueDict['15'] = 'Portuguese'
motherTongueDict['16'] = 'Punjabi (Panjabi)'
motherTongueDict['17'] = 'Spanish'
motherTongueDict['18'] = 'All other languages' #'Ukrainian'
motherTongueDict['19'] = 'All other languages' #'Vietnamese'
motherTongueDict['20'] = 'Austro-Asiatic languages'
motherTongueDict['21'] = 'Other European languages'
motherTongueDict['22'] = 'Russian'
motherTongueDict['23'] = 'All other languages' #'Other Slavic languages'
motherTongueDict['24'] = 'All other languages' #'Uralic languages'
motherTongueDict['25'] = 'Other Afro-Asiatic and African languages' #'Other Afro-Asiatic languages'
motherTongueDict['26'] = 'Tamil'
motherTongueDict['27'] = 'All other languages' #'Other Dravidian languages'
motherTongueDict['28'] = 'All other languages' #'Korean'
motherTongueDict['29'] = 'Other East and Southeast Asian languages'
motherTongueDict['30'] = 'Tagalog (Pilipino, Filipino)'
motherTongueDict['31'] = 'All other languages' #'Niger-Congo languages and other African languages'
motherTongueDict['32'] = 'All other languages' #'All other single languages'
motherTongueDict['88'] = 'Not available'

In [4]:
homeLanguageDict = {}
homeLanguageDict['1'] = 'No non-official languages'
homeLanguageDict['2'] = 'Aboriginal languages'
homeLanguageDict['3'] = 'Italian'
homeLanguageDict['4'] = 'Spanish'
homeLanguageDict['5'] = 'Portugese'
homeLanguageDict['6'] = 'German'
homeLanguageDict['7'] = 'Russian'
homeLanguageDict['8'] = 'Polish'
homeLanguageDict['9'] = 'Slavic languages'
homeLanguageDict['10'] = 'Other European languages'
homeLanguageDict['11'] = 'Arabic'
homeLanguageDict['12'] = 'Other Afro-Asiatic and African languages'
homeLanguageDict['13'] = 'Punjabi (Panjabi)'
homeLanguageDict['14'] = 'Urdu'
homeLanguageDict['15'] = 'Persian (Farsi)'
homeLanguageDict['16'] = 'Other Indo-Iranian languages'
homeLanguageDict['17'] = 'Cantonese'
homeLanguageDict['18'] = 'Mandarin'
homeLanguageDict['19'] = 'Chinese languages'
homeLanguageDict['20'] = 'Austro-Asiatic languages'
homeLanguageDict['21'] = 'Tagalog (Pilipino, Filipino)'
homeLanguageDict['22'] = 'Other East and Southeast Asian languages'
homeLanguageDict['23'] = 'Tamil'
homeLanguageDict['24'] = 'All other languages'
homeLanguageDict['88'] = 'Not available'

In [5]:
# dataset = dataset[dataset.HLANO != 88]
# dataset = dataset[dataset.MTNNO != 88]
# class label: home language part A - first language write in component
homeLang = dataset['HLANO']
# class label: mother tongue part A - first language write in component
motherTongue = dataset['MTNNO']
fields.remove('HLANO')
fields.remove('MTNNO')
homeLanguageMapped = homeLang.apply(lambda x: homeLanguageDict[str(x)])
motherTongueMapped = motherTongue.apply(lambda x: motherTongueDict[str(x)])
languageShift = homeLanguageMapped.ne(motherTongueMapped)

fields.remove('WEIGHT')
x = dataset[fields]
weights = dataset["WEIGHT"]

In [6]:
homeLanguageMapped

0         No non-official languages
1         No non-official languages
2         No non-official languages
3         No non-official languages
4         No non-official languages
5         No non-official languages
6         No non-official languages
7         No non-official languages
8         No non-official languages
9         No non-official languages
10        No non-official languages
11        No non-official languages
12        No non-official languages
13        No non-official languages
14        No non-official languages
15        No non-official languages
16        No non-official languages
17        No non-official languages
18        No non-official languages
19        No non-official languages
20        No non-official languages
21        No non-official languages
22        No non-official languages
23        No non-official languages
24        No non-official languages
25        No non-official languages
26        No non-official languages
27        No non-official la

In [7]:
motherTongueMapped

0            No non-official languages
1            No non-official languages
2            No non-official languages
3            No non-official languages
4            No non-official languages
5            No non-official languages
6            No non-official languages
7            No non-official languages
8            No non-official languages
9            No non-official languages
10           No non-official languages
11           No non-official languages
12           No non-official languages
13           No non-official languages
14           No non-official languages
15           No non-official languages
16           No non-official languages
17           No non-official languages
18           No non-official languages
19                           Cantonese
20           No non-official languages
21                            Mandarin
22           No non-official languages
23           No non-official languages
24           No non-official languages
25           No non-offic

In [8]:
languageShift

0         False
1         False
2         False
3         False
4         False
5         False
6         False
7         False
8         False
9         False
10        False
11        False
12        False
13        False
14        False
15        False
16        False
17        False
18        False
19         True
20        False
21         True
22        False
23        False
24        False
25        False
26        False
27        False
28        False
29        False
          ...  
930391    False
930392    False
930393    False
930394    False
930395    False
930396     True
930397    False
930398     True
930399    False
930400    False
930401    False
930402    False
930403    False
930404    False
930405    False
930406    False
930407     True
930408    False
930409    False
930410    False
930411    False
930412    False
930413    False
930414    False
930415    False
930416    False
930417    False
930418    False
930419    False
930420    False
Length: 930421, dtype: b

In [9]:
x

,PPSORT,ABOID,BFNMEMB,CFSTAT,Citizen,CMA,DETH123,HHTYPE,MarStH,PR,...,PresMortG,REPAIR,ROOMS,SSGRAD,Tenur,TotInc,TotInc_AT,VALUE,VisMin,WRKACT
0,453141,6,0,2,1,505,1,2,2,35,...,1,1,11,5,1,97000,73000,450000,13,11
1,923226,6,0,4,1,505,2,2,1,35,...,1,1,11,4,1,49071,39759,440000,13,11
2,385097,6,0,4,1,505,2,2,1,35,...,1,1,11,4,1,49071,39759,440000,13,11
3,732612,6,0,2,1,999,2,2,3,35,...,1,2,8,8,1,46000,41000,839779,13,11
4,143665,6,0,1,1,999,1,1,3,11,...,1,3,5,6,1,30000,26000,60000,13,9
5,459269,6,0,1,1,999,2,1,2,35,...,0,2,7,4,1,82000,69000,839779,13,1
6,632419,6,0,1,1,999,2,1,2,35,...,0,2,7,6,1,51000,45000,839779,13,1
7,284347,6,0,6,1,825,2,8,6,48,...,0,1,5,6,1,41000,37000,310000,13,1
8,52611,6,0,1,1,421,1,1,2,24,...,1,1,4,5,2,53000,43000,494190,13,11
9,36927,6,0,4,1,532,1,2,1,35,...,1,1,10,4,1,27000,26000,640000,13,7


In [10]:
weights

0         37.037277
1         37.037277
2         37.037277
3         37.037277
4         37.120914
5         37.037277
6         37.037277
7         37.019784
8         37.041550
9         37.037277
10        37.037277
11        37.037277
12        37.039118
13        37.037277
14        37.039118
15        37.039118
16        37.037277
17        37.037277
18        37.042280
19        37.042280
20        37.042280
21        37.037277
22        37.039118
23        37.037277
24        37.012589
25        37.041550
26        37.042280
27        37.042280
28        37.041550
29        37.042280
            ...    
930391    37.040742
930392    37.040742
930393    37.041550
930394    37.041550
930395    37.041550
930396    37.041550
930397    37.037277
930398    37.037277
930399    37.051222
930400    37.051222
930401    37.051222
930402    37.051222
930403    37.019784
930404    37.032779
930405    37.019784
930406    37.037277
930407    37.041550
930408    37.037277
930409    37.032779


In [11]:
from sklearn import datasets

In [12]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()

In [13]:
from sklearn.model_selection import train_test_split
weights_train, weights_test, x_train, x_test, languageShift_train, languageShift_test = train_test_split(weights, x, languageShift)

In [14]:
classifier.fit(x_train, languageShift_train, weights_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [15]:
languageShift_pred = classifier.predict(x_test)

In [16]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

The Confusion Matrix indicates, given the predicted language (row names), for how many records did the algorith match the actual language. The classification report and accuracy score are other ways to evaluate how well the classification algorithm performed.

In [17]:
confusionMatrixLanguageShift = pd.DataFrame(
    confusion_matrix(languageShift_test,languageShift_pred), 
    index=['False', 'True'], 
    columns=['False', 'True'], 
)

In [18]:
print(confusionMatrixLanguageShift)
print(classification_report(languageShift_test,languageShift_pred,target_names=['False', 'True']))
print(accuracy_score(languageShift_test, languageShift_pred))

        False  True
False  196357  8115
True    25451  2683
              precision    recall  f1-score   support

       False       0.89      0.96      0.92    204472
        True       0.25      0.10      0.14     28134

   micro avg       0.86      0.86      0.86    232606
   macro avg       0.57      0.53      0.53    232606
weighted avg       0.81      0.86      0.83    232606

0.855695897784236


In [19]:
!pip install tabulate

In [34]:
from tabulate import tabulate
importances = classifier.get_params()
print(importances)
#headers = ["name", "score"]
#values = sorted(zip(x_train.columns, classifier.feature_importances_), key=lambda x: x[1] * -1)
#print(tabulate(values, headers, tablefmt="plain"))

{'priors': None, 'var_smoothing': 1e-09}


In [33]:
#from sklearn.tree import export_graphviz
#estimator = classifier.estimators_[5]
# Creates dot file named randomForestTreeSample.dot
#export_graphviz(estimator, 
#                out_file='randomForestTreeSample.dot', 
#                feature_names = list(x.columns),
#                class_names = ['False', 'True'],
#                rounded = True, proportion = False, 
#                precision = 2, filled = True,
#                max_depth=5)